<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Build Your First Data Analyst Agent with LangChain and Teradata Enterprise MCP
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style="font-size:20px;font-family:Arial"><b>Introduction</b></p>

<p style="font-size:16px;font-family:Arial">
The Model Context Protocol (MCP) is an open standard that enables AI models to securely interact with local and remote resources through standardized server implementations. By combining MCP with LangChain, developers can build powerful AI agents that leverage enterprise data sources like Teradata.
</p>

<p style="font-size:16px;font-family:Arial">
This notebook demonstrates how to connect a LangChain agent to a Teradata Enterprise MCP, enabling natural language queries against your Teradata database.
</p>

---

<p style="font-size:20px;font-family:Arial"><b>Key Concepts</b></p>

<p style="font-size:18px;font-family:Arial"><b>What is MCP?</b></p>

<p style="font-size:16px;font-family:Arial">
The Model Context Protocol (MCP) provides a standardized way for AI applications to connect with external data sources and tools. It acts as a bridge between language models and enterprise systems.
</p>

<p style="font-size:18px;font-family:Arial"><b>What is LangChain?</b></p>

<p style="font-size:16px;font-family:Arial">
LangChain is a framework for developing applications powered by language models. It provides tools for building agents that can reason, plan, and execute tasks using various tools and data sources.
</p>

---

<p style="font-size:20px;font-family:Arial"><b>Architecture Overview</b></p>

<table style="font-size:16px; font-family:Arial; border-collapse:collapse; width:100%; background-color:#1e1e1e; border:1px solid #ccc;">
  <tr>
    <th style="border:1px solid #ccc; padding:8px; text-align:left;">Component</th>
    <th style="border:1px solid #ccc; padding:8px; text-align:left;">Description</th>
  </tr>
  <tr>
    <td style="border:1px solid #ccc; padding:8px;">LangChain Agent</td>
    <td style="border:1px solid #ccc; padding:8px;">Orchestrates reasoning and tool execution based on user queries</td>
  </tr>
  <tr>
    <td style="border:1px solid #ccc; padding:8px;">MCP Client</td>
    <td style="border:1px solid #ccc; padding:8px;">Connects to MCP servers and exposes tools to the agent</td>
  </tr>
  <tr>
    <td style="border:1px solid #ccc; padding:8px;">Teradata Enterprise MCP</td>
    <td style="border:1px solid #ccc; padding:8px;">Provides secure access to Teradata database operations</td>
  </tr>
  <tr>
    <td style="border:1px solid #ccc; padding:8px;">LLM (OpenAI)</td>
    <td style="border:1px solid #ccc; padding:8px;">Powers the agent's reasoning and natural language understanding</td>
  </tr>
</table>


<p style = 'font-size:20px;font-family:Arial'><b>What We Will Do in This Notebook</b></p>

<p style = 'font-size:16px;font-family:Arial'>
This notebook guides you through setting up a LangChain agent connected to a Teradata Enterprise MCP. By the end of this tutorial, you will be able to query your Teradata database using natural language. Here's what we'll cover:

</p>

<div style = 'font-size:16px;font-family:Arial'>
<ol>
<li><b>Installing necessary libraries</b></li>
<li><b>Importing required modules</b></li>
<li><b>Loading environment variables</b></li>
<li><b>Connecting to Teradata Enterprise MCP</b></li>
<li><b>Initializing the LLM and creating an agent</b></li>
<li><b>Querying the database with natural language</b></li>
</ol>
</div>


<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>1. Installing Necessary Libraries</b></p>

<p style = 'font-size:16px;font-family:Arial'>We need to install the LangChain MCP adapters along with the core LangChain and OpenAI integration packages. These libraries enable communication between our agent and the MCP server.</p>

In [ ]:
%%capture
pip install -U langchain-mcp-adapters langchain langchain-openai pip install httpx --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Note:</b><i> Please restart the kernel after executing the above cell to include/update these libraries into memory for this kernel. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i> and then clicking <b>Restart</b>.</p>
</div>

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>2. Importing Necessary Libraries</b></p>

<p style = 'font-size:16px;font-family:Arial'>Import the required modules for async operations, environment variable management, MCP client connectivity, and LangChain agent creation.</p>

In [ ]:
import asyncio, sys, os, httpx
import time
from dotenv import load_dotenv
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>3. Loading Credentials and Variables</b></p>

<p style = 'font-size:16px;font-family:Arial'>Read the database credentials and other required variables from our <code>.env</code> file. This file contains sensitive credentials including the LiteLLM API key required for connecting to the language model service.</p>

In [ ]:
environment_path = "/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"
load_dotenv(dotenv_path=environment_path) 
llm_key = os.getenv('litellm_key')
llm_url = os.getenv('litellm_base_url')
username = os.getenv('username'),
password = os.getenv('my_variable')

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>4. Connecting to Teradata Enterprise MCP Server</b></p>

<p style = 'font-size:16px;font-family:Arial'>Establish a connection to the Teradata Enterprise MCP Server using the <code>MultiServerMCPClient</code>. This client handles the HTTP transport and authentication required to communicate with the MCP server.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Connection Parameters:</b></p>
<ul style="font-size:16px;font-family:Arial">
  <li><b>transport:</b> HTTP protocol for communication</li>
  <li><b>url:</b> MCP server endpoint</li>
  <li><b>auth:</b> Basic authentication with username and password</li>
</ul>

In [ ]:
async def get_mcp_tools():
    client = MultiServerMCPClient(
        {
            "teradata": {
                "transport": "http",
                "url": "http://host.docker.internal:8001/mcp",
                "auth": httpx.BasicAuth("username","password")
            }
        }
    )
    tools = await client.get_tools()
    return client, tools

client, tools = await get_mcp_tools()
print([t.name for t in tools])

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>5. Initializing the LLM and Creating the Agent</b></p>

<p style = 'font-size:16px;font-family:Arial'>Initialize the language model and create a LangChain agent that can use the MCP tools. The agent acts as a data analyst, interpreting user questions and executing appropriate database queries.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Agent Configuration:</b></p>
<ul style="font-size:16px;font-family:Arial">
  <li><b>model:</b> OpenAI GPT-4.1 hosted via LiteLLM</li>
  <li><b>tools:</b> MCP tools retrieved from the Teradata server</li>
  <li><b>system_prompt:</b> Defines the agent's role as a data analyst</li>
</ul>

In [ ]:
llm = init_chat_model(
    model="openai-gpt-41",
    model_provider="openai",
    base_url=llm_url,
    api_key=llm_key,
)
agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=(''' 
    You are a data analyst that responds to user questions regarding data in a teradata system '''
    ),
)

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>6. Querying the Database</b></p>

<p style = 'font-size:16px;font-family:Arial'>Now we can interact with the Teradata database using natural language. The agent will interpret the question, determine which MCP tools to use, and return a meaningful response.</p>

<p style = 'font-size:18px;font-family:Arial'><b>6.1 Ask a question.</b></p>
<p style = 'font-size:18px;font-family:Arial'>In this example, you can ask the agent "What databases contain business information?".</p>

In [ ]:
user_query = input('What would you like to know about your data? ')
result = await agent.ainvoke(
    {"messages": [HumanMessage(content=user_query)]},
)

<hr style="height:1px;border:none;">
<p style="font-size:18px;font-family:Arial"><b>6.1 View the Agent Response</b></p>
<p style="font-size:16px;font-family:Arial">Display the final response from the agent. The agent processes the query through the MCP tools and returns a natural language answer about the available databases.</p>

In [ ]:
print(result["messages"][-1].content)

<footer style="padding-bottom:35px; border-bottom:3px solid">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025,2026. All Rights Reserved
        </div>
    </div>
</footer>